# WeatherPy
----

### Analysis
* As expected, the weather becomes significantly warmer as one approaches the equator (0 Deg. Latitude). More interestingly, however, is the fact that the southern hemisphere tends to be warmer this time of year than the northern hemisphere. This may be due to the tilt of the earth.
* There is no strong relationship between latitude and cloudiness. However, it is interesting to see that a strong band of cities sits at 0, 80, and 100% cloudiness.
* There is no strong relationship between latitude and wind speed. However, in northern hemispheres there is a flurry of cities with over 20 mph of wind.

---

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from pprint import pprint
import openweathermapy as owm
import urllib
import csv

# Import API key
from api_keys import api_key

#use time.sleep(5) for pausing between API requests so that we don't overload processes

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
#output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [2]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

#The code above will pick 1500 random latitude and 1500 random longitude coordinates 
#between the defined ranges and the closest city using numpy randomizer.  Distinct cities
#will be added to the list based on the (if city not in cities) condition.

#Just checking the output of lat/lng zip to understand data
#lat_lngs_df = pd.DataFrame(list(zip(lats, lngs)))
#lat_lngs_df

print(len(cities))

590


In [3]:
#Define base URL, units, and a query URL that adds on the API key and unit of measure.
#city added to URL later when looping through city list
url = "http://api.openweathermap.org/data/2.5/weather?"

#I use imperial for charting purposes.  There is a greater variance in numbers which should
#illustrate a more meaningful chart - also, I understand F better than C (temp)
units = "Imperial"
query_url = f"{url}appid={api_key}&units={units}&q="

#test output of JSON city requests
#print 3 results just for a preview
#use time.sleep() to pause between requests
counter = 0
for city in cities:
    response = requests.get(query_url + city).json()
    pprint(response)
    counter += 1
    time.sleep(1)
    if counter == 3:
        break

#I noticeded that there is a "city not found" error 
#{'cod': '404', 'message': 'city not found'}

{'base': 'stations',
 'clouds': {'all': 40},
 'cod': 200,
 'coord': {'lat': 47.66, 'lon': -52.73},
 'dt': 1537668000,
 'id': 6167817,
 'main': {'humidity': 81,
          'pressure': 1012,
          'temp': 48.2,
          'temp_max': 48.2,
          'temp_min': 48.2},
 'name': 'Torbay',
 'sys': {'country': 'CA',
         'id': 3467,
         'message': 0.0067,
         'sunrise': 1537694346,
         'sunset': 1537737989,
         'type': 1},
 'visibility': 24140,
 'weather': [{'description': 'scattered clouds',
              'icon': '03n',
              'id': 802,
              'main': 'Clouds'}],
 'wind': {'deg': 310, 'gust': 10.3, 'speed': 12.75}}
{'base': 'stations',
 'clouds': {'all': 8},
 'cod': 200,
 'coord': {'lat': 12.49, 'lon': -16.54},
 'dt': 1537671125,
 'id': 2246901,
 'main': {'grnd_level': 1024.37,
          'humidity': 99,
          'pressure': 1024.37,
          'sea_level': 1024.98,
          'temp': 81.17,
          'temp_max': 81.17,
          'temp_min': 81.17},
 '

In [4]:
#preview of constructed query URL + city
#URL = OK
print(query_url + city)

http://api.openweathermap.org/data/2.5/weather?appid=0c7809d55303f1644e7ea61b6ed8ea9c&units=Imperial&q=sola


In [ ]:
#!!!WARNING!!! RUNNING THIS BLOCK TAKES A LONG TIME

#prepare empty list to hold city info values
city_data = []

#for printing counts
limit = 0
set_var = 1
item_var = 1

##Perform API Calls
##Perform a weather check on each city using a series of successive API calls.
##Include a print log of each city as it'sbeing processed (with the city number and city name).

#loop through cities to extract information about all the ~500 random cities
#impose try/except for anomalies
for city in cities:
    try:
        # add city to query URL to get JSON data as tested above
        response = requests.get(query_url + city).json()
        city_name = response["name"]
        city_lat = response["coord"]["lat"]
        city_lng = response["coord"]["lon"]
        city_max_temp = response["main"]["temp_max"]
        city_humidity = response["main"]["humidity"]
        city_clouds = response["clouds"]["all"]
        city_wind = response["wind"]["speed"]
        city_country = response["sys"]["country"]
        city_date = response["dt"]
        # Append the City information into city_data list
        city_data.append({"City": city_name, 
                          "Latitude": city_lat, 
                          "Longitude": city_lng, 
                          "Max Temperature": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})
        if item_var < 51:
            print(f'{city} (Set: {set_var}, Item: {item_var})') 
            item_var += 1
        else:
            set_var += 1
            item_var = 1
        time.sleep(1)
    # If city not found, skip to next record
    except:
        print(f'****{city} not found.  Skipping...')
#    limit += 1
#    if limit == 600:
#        break

torbay (Set: 1, Item: 1)
oussouye (Set: 1, Item: 2)
sola (Set: 1, Item: 3)
****illoqqortoormiut not found.  Skipping...
albany (Set: 1, Item: 4)
marquard (Set: 1, Item: 5)
cape town (Set: 1, Item: 6)
rikitea (Set: 1, Item: 7)
****amderma not found.  Skipping...
upernavik (Set: 1, Item: 8)
ushuaia (Set: 1, Item: 9)
barrow (Set: 1, Item: 10)
adrar (Set: 1, Item: 11)
arraial do cabo (Set: 1, Item: 12)
vao (Set: 1, Item: 13)
genhe (Set: 1, Item: 14)
bambous virieux (Set: 1, Item: 15)
jacareacanga (Set: 1, Item: 16)
kapaa (Set: 1, Item: 17)
vaini (Set: 1, Item: 18)
****haibowan not found.  Skipping...
plouzane (Set: 1, Item: 19)
atuona (Set: 1, Item: 20)
jamestown (Set: 1, Item: 21)
kloulklubed (Set: 1, Item: 22)
bilma (Set: 1, Item: 23)
****attawapiskat not found.  Skipping...
busselton (Set: 1, Item: 24)
****taolanaro not found.  Skipping...
punta arenas (Set: 1, Item: 25)
****barentsburg not found.  Skipping...
goure (Set: 1, Item: 26)
****mys shmidta not found.  Skipping...
vila franca 

In [ ]:
# Create CSV file
with open('output_data/cities.csv', 'w', newline='') as csvfile:
     fields = ['City', 'Latitude', 'Longitude', 'Max Temperature', 'Humidity', 'Cloudiness', 'Wind Speed', 'Country', 'Date']
     writer = csv.DictWriter(csvfile, fieldnames=fields)
     writer.writeheader()
     for x in city_data:
         writer.writerow(x)
            
print(csvfile)

In [ ]:
#read CSV and output to dataframe
cities_df = pd.read_csv("output_data/cities.csv")
cities_df.head()

### Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

#### Latitude vs. Temperature Plot

In [ ]:
# Scatterplot: Latituded vs. Max Temperature
cities_df.plot(kind="scatter", 
               x="Latitude", 
               y="Max Temperature", 
               grid=True, 
               figsize=(20,10),
               title="Latitude vs Max Temperature")

plt.savefig('output_data/latitude_temperature.png')
plt.show()

#### Latitude vs. Humidity Plot

In [ ]:
# Scatterplot: Latituded vs. Humidity
cities_df.plot(kind="scatter", 
               x="Latitude", 
               y="Humidity", 
               grid=True, 
               figsize=(20,10),
               title="Latitude vs Humidity")

plt.savefig('output_data/latitude_huidity.png')
plt.show()

#### Latitude vs. Cloudiness Plot

In [ ]:
# Scatterplot: Latituded vs. Cloudiness
cities_df.plot(kind="scatter", 
               x="Latitude", 
               y="Cloudiness", 
               grid=True, 
               figsize=(20,10),
               title="Latitude vs Cloudiness")

plt.savefig('output_data/latitude_cloudiness.png')
plt.show()

#### Latitude vs. Wind Speed Plot

In [ ]:
# Scatterplot: Latituded vs. Wind Speed
cities_df.plot(kind="scatter", 
               x="Latitude", 
               y="Wind Speed", 
               grid=True, 
               figsize=(20,10),
               title="Latitude vs Wind Speed")

plt.savefig('output_data/latitude_windspeed.png')
plt.show()